<div style="background-color: #ffffff; color: #000000; padding: 10px;">
<img src="../media/img/kisz_logo.png" width="192" height="69"> 
<h1> NLP Fundamentals
<h2> Working with Embeddings
</div>

<div style="background-color: #f6a800; color: #ffffff; padding: 10px;">
<h2>Part 2.3 - Term Frequency-Inverse Document Frequency (TF-IDF)
</div>

Here we are going to build a improved better version of Bag of Words, the *Term Frequency - Inverse Document Frequency* model, and we will do also from scratch and with the <kbd>gensim</kbd> library.

Let's start with some imports.

In [ ]:
# imports
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import nb_config

from src.data import data_loader

import warnings
warnings.filterwarnings('ignore')

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>1. Overview
</div>

TF-IDF (Term Frequency-Inverse Document Frequency) is a text analysis model used to assess the importance of words in documents within a collection. It combines two metrics: Term Frequency (TF), which measures how often a term appears in a document, and Inverse Document Frequency (IDF), which gauges the rarity of a term across the entire document collection. The TF-IDF score for a term in a document is calculated by multiplying its TF by its IDF, providing a numerical representation of the term's significance in the context of the entire corpus.

To build a TF-IDF model, we need to follow this steps:
1. Create the vocabulary. Well, nothing new. We did it already for the Bag of Word models.
2. Calculate the Term Frequency (TF) for each term in a document by dividing the number of occurrences of the term by the total number of terms in that document. It is almost the samenas a Bag of Words, but with relative frequencies instead of absolute ones.
   $$TF=\frac{\textit{Number of occurrences of term in document}}{\textit{Total number of terms in document}}​$$
3. Compute the Inverse Document Frequency (IDF) for each term by taking the logarithm of the ratio of the total number of documents to the number of documents containing the term.
   $$IDF=\log{\frac{\textit{Total number of documents}​}{\textit{Number of documents containing the term}}}​$$
4. Obtain the TF-IDF score for each term in a document by multiplying its TF by its IDF. The resulting scores provide a weighted representation of the terms across the documents in our corpus.
   $$\textit{TF-IDF}=TF×IDF$$

> <details>
> <summary>About the IDF formula</summary>
> We have said that the formula for IDF is the (common) logarithm of the ratio between the total number of documents and the number of documents containing the term, but that's not completely accurated. That's the usual formula, but there are several variants used in specific implementations, so you should never give for granted which formula was used for calculating it.
> </details>

TF-IDF models are widely used in information retrieval systems to rank documents based on their relevance to a given query. By assigning numerical weights to terms that capture both local and global information, TF-IDF helps identify key terms that characterize the content of a document and facilitates effective document matching and retrieval.

But for that we are going to need the data. Let's prepare it.

 


In [ ]:
df, params = data_loader("my_tokenized_data.parquet")

# extract the arguments for the normalize function
tokenizer = params['tokenizer']
arguments = params['args']

# print the tokenizer used getting the tokens
print(f"Tokenizer used: {tokenizer}")
print(f"Arguments passed to the normalize function:\n{arguments}")

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>2. Creating the vocabulary
</div>

We need to build the vocabulary from our corpus of documents, but we know already how to do that from the last part, so we will just go quick through it.

In [ ]:
# create an easy access to the tokens
tokens = df['tokens']

# create an empty set
vocabulary = set()

# iterate over the pd Series and updates the set
for index in df.index:
   vocabulary.update(df.loc[index, 'tokens'].tolist())

# turn the vocabulary into a list
vocabulary = list(vocabulary)

# print vocabulary size
print(f"The vocabulary has {len(vocabulary)} tokens.")

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>3. Getting the TF-IDF vectors
</div>

Now it is when things start getting interesting. We are going to create the tf-idf representations of our documents. But let's approach the problem in small steps

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

Create a sparse representation of the token frequency for each document. Follow the next steps:
1. Create a sparse matrix for the corpus of documents and populate it with the frequencies as we did for the Bag of Words.
2. Count the number of tokens in each document and divide the sparse matrix by that vector. We have now the TF matrix. Store it in the variable <kbd>Ttf_df</kbd>. 
</div>

<br>

This part is going to be almost like coding a Term Frequency Bag of Words, with a small twist. You have done that already before, so we will write for you already the first lines of code.

> <details>
> <summary>Dividing sparse matrices</summary>
> It could happen that you try to multiply or divide matrices and vectors and you find strange behaviour. Don't worry it is normal. It has to do with how the objects that define the sparse matrices or vectors have been implemented, and you could have problems in *SciPy* but also in *pandas*. We recommend you to work better with pandas and use the method <kbd>.div()</kbd> instead of trying the usual operator <kbd>/</kbd>.
> 
> </details>

<br>

> <details>
> <summary>Is it going too slow?</summary>
> Well, the first part is almost identical to what you did in the Bag of Words. Go there and take a look...
> 
> </details>

<br>

> <details>
> <summary>Here is a possible solution</summary>
> You know? Just in case...
>
> <pre><code># populate the matrix
> # remember: it works like a TF BoW, not a binary BoW
> for i, string_list in enumerate(tokens):
>    for string in string_list:
>        freq_sparse[i, string_to_index[string]] += 1
> 
> # convert the DOK matrix to a sparse DataFrame
> freq_sparse = pd.DataFrame.sparse.from_spmatrix(freq_sparse, columns=vocabulary)
> 
> # 2. create the TF 
> # count the tokens per document
> ntokens = tokens.apply(lambda x: len(x))
> 
> # calculate the tf matrix
> tf_df = freq_sparse.div(ntokens, axis=0)
> 
> </code></pre>
> </details>

In [ ]:
# import dok_matrix
from scipy.sparse import dok_matrix

# 1. calculate the frequency sparse matrix
# create the sparse matrix
freq_sparse = dok_matrix((len(tokens), len(vocabulary)), dtype=np.int32)
 
# dictionary with pairs 'token':index
string_to_index = {string: i for i, string in enumerate(vocabulary)}

# populate the matrix
# remember: it works like a TF BoW, not a binary BoW
for ...:
    for ...:
        ...

# convert the DOK matrix to a sparse DataFrame
freq_sparse = ...

# 2. create the TF 
# count the tokens per document
ntokens = ...

# calculate the tf matrix
tf_df = ...

It's time to get Inverse Document Frequency vector.

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

Create the Inverse Document Frequency vector. Follow the next steps:
1. Calculate the number of documents. You can extract that information from the <kbd>tf_df</kbd> dataframe. Store it in a variable called <kbd>ndocs</kbd>.
2. Calculate in how many docuemnts appear each token and store it as a pandas series named <kbd>docfreq</kbd>.
3. Create a pandas Series called <kbd>tfidf_s</kbd>that contains the logarithm of the ratio of <kbd>ndocs</kbd> and <kbd>docfreq</kbd>>.
</div>


This exercise should be easier than the last one.

> <details>
> <summary>But if you get stuck</summary>
> You could use this...
>
> <pre><code># calculate the document count 
> ndocs = tf_df.shape[0]
> 
> # calculate the document frequency
> docfreq = tf_df.astype(bool).sum(axis=0)
> 
> # calculate the  logarithm of their ratio
> idf_s = np.log(ndocs / docfreq)
> 
> </code></pre>
> </details>

In [ ]:
# calculate the document count 
ndocs = ...

# calculate the document frequency
docfreq = ...

# calculate the logarithm of their ratio
idf_s = ...

All that remains is to multiply the term frequency matrix and the inverse document frequency vector to get our tf-idf model.

In [ ]:
tfidf_df = tf_df * idf_s

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>4. Making queries
</div>

Time to test how this model performs. We are going to make a query and see which films are more similar to our query. For easy comparing we will start with the same query we used for Bag of Words. We want to get recommended *The Shawshank Redemption* so we write this query:

> *An innocent man goes to prison accused of killing his wife and her lover, but never loses the hope*

We will pass that sentence through a tokenizer. We will use the metadata we collected before and create a tokenizer from scratch with that metadata, so we will have exactly the same tokenizer we used for the rest of the texts.

In [ ]:
query = "An innocent man in prison that never loses the hope starts helping the warden as accountant"

print(tokenizer)
print(arguments)

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

Create a vector representation of the query. We will follow this steps:
1. Tokenize the query as usual
2. Make a list with all the unique tokens in the query and store it in the variable <kbd>query_tokens</kbd>
3. Create and populate a scipy sparse matrix as we just did for the corpus vectors.
4. Turn the sparse matrix into a pandas Dataframe.
5. Store in the variable <kbd>query_ntokens</kbd> the number of tokens in the query.
6. Calculate <kbd>query_tf</kbd>, dividing the vector in <kbd>query_df</kbd> by <kbd>query_ntokens</kbd>.
7. Calculate the TF-IDF vector for the query, <kbd>query_tfidf</kbd> multiplying the query dataframe and the corpus idf vector.
</div>

You have repeated the first steps already many times and we are quite sure you know already how it goes, so the code is already there for you. I know, you are welcome!

> <details>
> <summary>We are sure you don't need the solution, but just in case...</summary>
> 
> There you go. This code should work:
> 
> <pre><code>from src.normalizing import normalize, NLTKTokenizer  # <- import the tokenizer you need
>  
> # instantiate the tokenizer
> tkn = NLTKTokenizer()
> 
> # tokenize the query with the same tokenizer you used for the corpus texts
> query_tokens = list(set(normalize(text=query, tkn=tkn, **arguments)[1]))
> 
> # create the sparse matrix
> query_sparse = dok_matrix((1, len(vocabulary)), dtype=np.int32)
> 
> # populate the matrix
> for string in query_tokens:
>     if string in vocabulary:
>         query_sparse[0, string_to_index[string]] = 1
> </code></pre>
> </details>

In [ ]:
from src.normalizing import normalize, ... # <- import the tokenizer you need

# instantiate the tokenizer
tkn = ...

# tokenize the query with the same tokenizer you used for the corpus texts
query_tokens = normalize(query, tkn, punct_signs=True)[1]

# create the sparse matrix
query_sparse = dok_matrix((1, len(vocabulary)), dtype=np.int32)

# populate the matrix
for string in query_tokens:
    if string in vocabulary:
        query_sparse[0, string_to_index[string]] += 1

# Convert the DOK matrix to a sparse DataFrame
query_df = pd.DataFrame.sparse.from_spmatrix(query_sparse, columns=vocabulary)

# count the tokens in the query
query_ntokens = ...

# calculate the tf matrix
query_tf = ...

# calculate the tfidf matrix
query_tfidf = ...

We are going to get the arrays out of our pandas dataframes and get the metrics as we did for our Bag of Words model.

In [ ]:
# getting the numpy ndarrays from our dataframes
query_vector = query_tfidf.to_numpy().flatten()
tfidf_matrix = tfidf_df.to_numpy()

# calculating the distances
euclid_distances = np.linalg.norm(tfidf_matrix - query_vector, axis=1)
dotprod_similarities = np.dot(tfidf_matrix, query_vector.T)
cos_similarities = cosine_similarity([query_vector], tfidf_matrix).flatten()

# creating the new dataframe and adding the extra columns
results = df.loc[:, ['title', 'descriptor']].copy()
results.loc[:, 'euclid_dist'] = euclid_distances
results.loc[:, 'dot_prod_sim'] = dotprod_similarities
results.loc[:, 'cos_sim'] = cos_similarities
results.loc[:, 'common_tokens'] = df.loc[:, 'tokens'].map(lambda x: list(set(x).intersection(query_tokens)))

In [ ]:
tfidf_matrix.shape

We can now check the best results by sorting the dataframe by metric.

In [ ]:
# choose the metric! options:
# 'euclid_dist', 'dot_prod_sim', 'cos_sim'
metric = 'cos_sim'

# choose number of results
n = 10

# show the results
results.sort_values(by=metric, ascending=False).head(n)

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

Feel free to look for other films using your own queries, or just let the model recommend you films based on what you would like to see. And try to answer this questions:
- How good are the recommendations?
- How close are the suggested films to what you were loooking for?
- Which metrics perform better for this model?
</div>

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>5. (Optional) TF-IDF with the <kbd>gensim</kbd> package
</div>

Once again, we could do the same in a more efficient way with libraries specifically developed to deal with texts. We have used <kbd>gensim</kbd> for Bag of Words models and we will work with it also for TF-IDF. Basically, the process is identical but before indexing we pass the corpus and the query through a class that computates the TF-IDF weights.

> <details>
> <summary>More options</summary>
> Yes, there are many, many more options. One specially interesting can be using the <kbd>TfidfVectorizer</kbd> from *SciKit-Learn*. But careful! *SciKit-Learn* uses a different formula for calculating the IDF.
> 
> You can find a good example of how to use it [here](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/03-TF-IDF-Scikit-Learn.html).
> </details>

As the dictionary is going to be identical to the one we already created for BoW, we will recreate it, but recover the one we saved when working with BoW.

In [ ]:
from gensim import corpora
from gensim.models import TfidfModel

# load the saved dictionary
dictionary = corpora.Dictionary.load('../artifacts/descriptors.dict')

# create the corpus with a vector for each document
corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in tokens]

# instantiate and fit the model with the corpus
model = TfidfModel(corpus)

In [ ]:
from gensim.similarities import Similarity
from gensim.test.utils import get_tmpfile

# get the path to the temporary file 'index'
index_tmpfile = get_tmpfile("index")

# create the index instantiating the class Similarity
# this time we don't pass the corpus as parameter, we
# pass the result of passing the corpus trough the model
index = Similarity(index_tmpfile, model[corpus], num_features=len(dictionary))

In our next step, we write down our query and tokenize it as we have done before. Then it needs to be vectorized.

We can then use our index to find the cosine similarity between the query and all the rest of descriptors in the corpus.

Finally we add the cosine similarities to a column in a copy of the original dataframe. 

In [ ]:
query = "An innocent man in prison that never loses the hope starts helping the warden as accountant"

# normalizing the query
query_tokens = normalize(query, tkn, punct_signs=True)[1]
query_bow = dictionary.doc2bow(query_tokens)
query_tfidf = model[query_bow]

# get cosine similarity between
# the query and all index documents
cos_similarities = index[query_tfidf]  # get similarities between the query and all index documents

# creating the dataframe
results = df.loc[:, ['title', 'descriptor']].copy()
results.loc[:, 'cos_sim'] = cos_similarities
results.loc[:, 'common_tokens'] = df.loc[:, 'tokens'].map(lambda x: list(set(x).intersection(query_tokens)))

# show the first ten results
results.sort_values(by='cos_sim', ascending=False).head(10)

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

As in the previous chapter, feel free to look for other films using your own queries, or just let the system recommend you films based on what you would like to see. And try to answer this questions:
- How good are the recommendations from this TF-IDF model created with <kbd>gensim</kbd>?
- Are they better than the ones we coded ourselves?
</div>

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>6. Advantages and disadvantages of TF-IDF Models
</div>

Let's discuss the pros and cons of TF-IDF models, specially compared with BoW models, and explore their applications.

#### Advantages:

> - **Term Weighting**: TF-IDF models incorporate term weighting, assigning importance to words based on their frequency in a document and their rarity across the entire corpus. This approach helps mitigate the issue of common words dominating the representation, allowing the model to focus on terms that are more discriminative and informative.
>
> - **Discriminative Power**: By giving higher weights to terms that are less common in the entire dataset but frequent in a specific document, TF-IDF models enhance the discriminative power of the representation. This is particularly valuable in tasks where identifying unique and relevant terms is crucial, such as document clustering or information retrieval.
>
> - **Flexibility in Term Importance**: TF-IDF allows for flexibility in defining the importance of terms. The model can be customized to give more weight to certain terms or domains, allowing users to adapt the representation to the specific requirements of their application.
> 
> - **Reduced Sensitivity to Stop Words**: Unlike BoW models, TF-IDF models are less sensitive to common stop words that appear frequently across documents. The inverse document frequency component helps down-weigh these common terms, preventing them from dominating the representation.

#### Disadvantages:

> - **Loss of Word Order and Semantics**: Similar to BoW models, TF-IDF models do not capture word order or semantic relationships between words. They represent documents as bags of weighted words, neglecting the sequential and contextual information present in the text.
>
> - **Difficulty in Handling Short and Noisy Documents**: TF-IDF models may face challenges when dealing with short documents or documents containing noise. In such cases, the limited context provided by shorter texts or the presence of irrelevant terms can lead to less effective representations. The model may struggle to discern the significance of terms in the absence of sufficient contextual information.
>
> - **Sparse Representations**: In high-dimensional spaces, TF-IDF vectors can also become sparse, posing challenges in terms of memory efficiency. The sparsity may require additional computational resources for storage and processing.
>
> - **Sensitivity to Vocabulary Changes**: TF-IDF models are sensitive to changes in the vocabulary. When the vocabulary of the corpus evolves or when new terms are introduced, the model may require updates or retraining to adapt, making it less robust to dynamic and evolving datasets. This sensitivity can impact the model's performance in real-world applications where the language evolves over time.

#### Applications:

> - **Information Retrieval**: TF-IDF models are widely used in information retrieval systems, similar to BoW models. In this context, TF-IDF vectors are employed to calculate the similarity between documents and queries, as we have done. The weighted representation allows the model to focus on terms that are discriminative for a given query, improving the relevance of retrieved documents.
>
> - **Document Clustering**: In document clustering tasks, where the goal is to group similar documents together, TF-IDF models prove effective. The weighted representation helps identify terms that are characteristic of specific clusters, facilitating the grouping of documents based on their content.
>
> - **Keyword Extraction**: TF-IDF models are useful for keyword extraction, where the goal is to identify the most important terms in a document. By considering both term frequency and document rarity, TF-IDF can highlight terms that are both frequent within a document and unique across the entire corpus.
>
> - **Content-Based Filtering**: In recommendation systems, TF-IDF models can be employed for content-based filtering. By representing items (e.g., articles, products) using TF-IDF vectors, the system can recommend items that are semantically similar to those a user has shown interest in, based on the content of the items.

These applications showcase how TF-IDF models, with their emphasis on term importance and discriminative power, are valuable in various information processing tasks.